# Bird Species Classier 
Built using the [Kaggle dataset](https://www.kaggle.com/gpiosenka/100-bird-species) by [Gerald Piosenka](https://www.kaggle.com/gpiosenka)

Project By: [Uzair](https://github.com/Uzair05)

## Download, extract and arrange data.

In [1]:
#!pip install --upgrade kaggle
!kaggle datasets download -d gpiosenka/100-bird-species

^C


In [ ]:
import os 
import zipfile

try:
    os.mkdir("./data/")
except Exception as err:
    print(f"Error in creating data/:\t{err}")


localzip = "./100-bird-species.zip"
zip_ref = zipfile.ZipFile(localzip, 'r')
try:
    zip_ref.extractall('data/')
except Exception as err:
    print(f"Erorr in zip extraction:\t{err}")
finally:
    zip_ref.close()

## Handle Data
Create train and test data.

In [26]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop

In [19]:
train_dir = "./data/train"
test_dir = "./data/test"
valid_dir = "./data/valid"

train_datagen = ImageDataGenerator(
    rescale = 1./255., 
    rotation_range = 40, 
    shear_range = 0.2,
    width_shift_range = 0.2, 
    height_shift_range = 0.2, 
    zoom_range = 0.2, 
    horizontal_flip = True, 
    vertical_flip = True, 
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)
valid_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [20]:
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size=(128,128), 
    batch_size=20, 
    class_mode="categorical", 
)
test_generator = test_datagen.flow_from_directory(
    test_dir, 
    target_size=(128,128), 
    class_mode="categorical", 
    batch_size=20
)
valid_generator = valid_datagen.flow_from_directory(
    valid_dir, 
    target_size=(128,128), 
    class_mode="categorical", 
    batch_size=20
)

Found 45980 images belonging to 315 classes.
Found 1575 images belonging to 315 classes.
Found 1575 images belonging to 315 classes.


## CNN Using Transfer Learning

In [21]:
from tensorflow.keras.applications.efficientnet import EfficientNetB3
# Import pretrained wights
#local_weights = "./data/EfficientNetB3-birds-98.92.h5"
pretrained_model = EfficientNetB3(
    input_shape=(128,128,3), 
    weights="imagenet", 
    include_top=False, 
)


43950080/43941136 [==============================] - 37s 1us/step


In [22]:
for layer in pretrained_model.layers:
    layer.trainable = False

In [ ]:
pretrained_model.summary()

In [28]:
last_layer = pretrained_model.get_layer('top_activation')
last_output = last_layer.output

x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(16, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(315, activation='softmax')(x)

model = tf.keras.Model(pretrained_model.input, x)

model.compile(optimizer = "rmsprop", 
              loss = 'categorical_crossentropy', 
              metrics = ['accuracy'])

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        accuracy = 0.95 # Percentage Accuracy.
        if(logs.get('accuracy') != None) and (logs.get('accuracy') >= accuracy): # Experiment with changing this value
          print(f"\nReached {accuracy*100}% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [ ]:
history = model.fit(
    train_generator,
    epochs = 50, 
    #steps_per_epoch = 50, 
    validation_data = test_generator, 
    verbose = 1, 
    validation_steps = 10,
    callbacks=[callbacks]
)

In [ ]:
history.history

In [ ]:
eval = model.evaluate(
    valid_generator,
    batch_size=20
)